### Assignment 2.1

Welcome to your first assignment! For this assignment, I want you to write a program that converts the string __snakety snake snake__ into __SnAkEtY_SnAkE_SnAkE__. Each word in the new string starts with an upper case letter, and alternates upper and lower case. Each space is replaced by an underscore. When I run the program, it should print the second string. Please just submit a single file Python 3 program structured using the `if __name__ == 'main':` structure I described.

In [1]:
def function():
    old = 'snakety snake snake'

    new = ''
    for c in range(len(old)):
        if not c % 2:
            new = new + old[c].upper()
        else:
            new = new + old[c].lower()
    print(new.replace(" ", "_"))

if __name__ == '__main__':
        function()

SnAkEtY_SnAkE_SnAkE


In [193]:
import itertools as it
import random as r

letters = list(map(chr, range(65, 91)))
letters.append(' ')

plaintext = input("Please enter your plaintext: ")
# plaintext = "macaroni"

### Generate salt
salt = ''
random_salt_length = r.randint(32,32)
while len(salt) != random_salt_length:
    salt += r.choice(letters)

### Implement Caesar cipher
spaces_to_shift = 17
shifted_letters = [
    letters[(idx + spaces_to_shift) % len(letters)] 
    for idx, x 
    in enumerate(letters)
]

def encipher_decipher(message, decryption_key, encryption_key):
    upper_plaintext = plaintext.upper()
    upper_salt = salt.upper()
    
    print('Plaintext message: {}'.format(''.join(plaintext)))

    ciphertext = [encryption_key[x] for x in upper_salt + upper_plaintext + upper_salt]
    print('Encrypted message: {}'.format(''.join(ciphertext)))

    decrypted_plaintext = [decryption_key[x] for x in ciphertext[len(upper_salt):len(upper_plaintext + upper_salt)]]
    print('Decrypted message: {}'.format(''.join(decrypted_plaintext)))
    # print("({}) {} : {}".format(plaintext, ''.join(ciphertext), ''.join(decrypted_plaintext)))

def create_keys(symbols, key_symbols):
    return (
        dict(it.zip_longest(symbols, key_symbols)),
        dict(it.zip_longest(key_symbols, symbols))
    )

encryption_key, decryption_key = create_keys(symbols=letters, key_symbols=shifted_letters)

if __name__ == '__main__':
    encipher_decipher(message=plaintext, decryption_key=decryption_key, encryption_key=encryption_key)

Plaintext message: testing
Encrypted message: HVOCPPINEXEY FMOJDKPUXQIWLPGPXNCJVIJZDXHVOCPPINEXEY FMOJDKPUXQIWLPGPXNC
Decrypted message: TESTING


In [15]:
import simple_ciphers as sc

def apply_cipher(cipher, plaintext, iv=''):
    plaintext = plaintext.upper().strip().replace(' ', '')
    
    
    slices = [plaintext[i:i+5] for i in range(0, len(plaintext), 5)]
    slices = [slice if len(slice) == 5 else slice + 'X' * (5 - len(slice)) for slice in slices]
    
    # enciphered_iv = sc.Vigenere(iv.upper())
    enciphered_iv = sc.Vigenere(iv.upper()).encipher(slices[0])
    
    c_1 = cipher.encipher(sc.Vigenere(enciphered_iv).encipher(slices[0]))
    slices = slices[1:len(slices)]
    enciphered_slices = []
    enciphered_slices.append(c_1)
    enciphered_slice = c_1
    for i, slice in enumerate(slices):
        # Encipher a new slice using the ciphertext of the previous operation
        # as the key to the embedded cipher
        enciphered_slice = cipher.encipher(sc.Vigenere(enciphered_slice).encipher(slice))
        
        # Append the newly enciphered slice
        enciphered_slices.append(enciphered_slice)
    
    
    #enciphered_slices = [cipher.encipher(iv + slice) for slice in slices]
    print('<=== ({})'.format(type(cipher)))
    print('Blocked ciphertext\t {}'.format(''.join(enciphered_slices)))
    print('Non-blocked ciphertext\t {}'.format(cipher.encipher(plaintext)))
    print('===>')

plaintext = 'This is a sooper sekkrit message'
vigenere = sc.Vigenere('iskey')
bifid = sc.Bifid()

iv = 'ab'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)

iv = 'cd'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)


<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 UHAPOUZCHARVDDCJXEPTDTGLRRPNMM
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 PHMQVPABEMAZPGQKQITMIAPFZFGWME
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>
<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 WJCRQWBEJCTXFFELZGRVFVINTTRPOO
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 XCRGTGIPNFRQUTNBNHLLLVNMXSKWIU
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>


In [20]:
# This is an implementation of the Output Feedback mode which sends the enciphered
# IV output as feedback for subsequent slices. This means the IV gets re-enciphered
# each slice and does not get used twice. Example block diagram show here:
# https://media.geeksforgeeks.org/wp-content/uploads/output-feedback-mode.png

import simple_ciphers as sc

def apply_cipher(cipher, plaintext, iv=''):
    plaintext = plaintext.upper().strip().replace(' ', '')
    
    
    slices = [plaintext[i:i+5] for i in range(0, len(plaintext), 5)]
    slices = [slice if len(slice) == 5 else slice + 'X' * (5 - len(slice)) for slice in slices]
    
    # Encipher the IV and use it as feedback for the next slice
    enciphered_iv = cipher.encipher(iv.upper())
    
    # Encipher the plaintext block with the ciphered IV so that essentially the IV is
    # re-enciphered each time and never use the same IV twice.
    enciphered_slices = [cipher.encipher(sc.Vigenere(enciphered_iv).encipher(slice)) for slice in slices]
    
    print('<=== ({})'.format(type(cipher)))
    print('Blocked ciphertext\t {}'.format(''.join(enciphered_slices)))
    print('Non-blocked ciphertext\t {}'.format(cipher.encipher(plaintext)))
    print('===>')

plaintext = 'This is a sooper sekkrit message'
vigenere = sc.Vigenere('iskey')
bifid = sc.Bifid()

iv = 'ab'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)

iv = 'cd'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)


<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 JSAPOILKLUFPJPKAVJFZCPKPGWPPUD
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 PSSYVFDBAHCLCPRDNGWTCXNHZBDXMZ
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>
<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 LUCRQKNMNWHRLRMCXLHBERMRIYRRWF
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 DOOYDAEIKIXPZNKKPFIRGECNDTSCLX
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>
